In [31]:
import pandas as pd
import torch

In [2]:
def load_activity_map():
    map = {}
    map[0] = 'transient'
    map[1] = 'lying'
    map[2] = 'sitting'
    map[3] = 'standing'
    map[4] = 'walking'
    map[5] = 'running'
    map[6] = 'cycling'
    map[7] = 'Nordic_walking'
    map[9] = 'watching_TV'
    map[10] = 'computer_work'
    map[11] = 'car driving'
    map[12] = 'ascending_stairs'
    map[13] = 'descending_stairs'
    map[16] = 'vacuum_cleaning'
    map[17] = 'ironing'
    map[18] = 'folding_laundry'
    map[19] = 'house_cleaning'
    map[20] = 'playing_soccer'
    map[24] = 'rope_jumping'
    return map

In [3]:
def generate_three_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    return [x,y,z]

def generate_four_IMU(name):
    x = name +'_x'
    y = name +'_y'
    z = name +'_z'
    w = name +'_w'
    return [x,y,z,w]

def generate_cols_IMU(name):
    # temp
    temp = name+'_temperature'
    output = [temp]
    # acceleration 16
    acceleration16 = name+'_3D_acceleration_16'
    acceleration16 = generate_three_IMU(acceleration16)
    output.extend(acceleration16)
    # acceleration 6
    acceleration6 = name+'_3D_acceleration_6'
    acceleration6 = generate_three_IMU(acceleration6)
    output.extend(acceleration6)
    # gyroscope
    gyroscope = name+'_3D_gyroscope'
    gyroscope = generate_three_IMU(gyroscope)
    output.extend(gyroscope)
    # magnometer
    magnometer = name+'_3D_magnetometer'
    magnometer = generate_three_IMU(magnometer)
    output.extend(magnometer)
    # oreintation
    oreintation = name+'_4D_orientation'
    oreintation = generate_four_IMU(oreintation)
    output.extend(oreintation)
    return output

def load_IMU():
    output = ['time_stamp','activity_id', 'heart_rate']
    hand = 'hand'
    hand = generate_cols_IMU(hand)
    output.extend(hand)
    chest = 'chest'
    chest = generate_cols_IMU(chest)
    output.extend(chest)
    ankle = 'ankle'
    ankle = generate_cols_IMU(ankle)
    output.extend(ankle)
    return output
    
def load_subjects(root='./PAMAP2_Dataset/Protocol/subject'):
    output = pd.DataFrame()
    cols = load_IMU()
    
    for i in range(101,110):
        path = root + str(i) +'.dat'
        subject = pd.read_table(path, header=None, sep='\s+')
        subject.columns = cols 
        subject['id'] = i
        output = output.append(subject, ignore_index=True)
    output.reset_index(drop=True, inplace=True)
    return output

data = load_subjects()

In [4]:
data

,time_stamp,activity_id,heart_rate,hand_temperature,hand_3D_acceleration_16_x,hand_3D_acceleration_16_y,hand_3D_acceleration_16_z,hand_3D_acceleration_6_x,hand_3D_acceleration_6_y,hand_3D_acceleration_6_z,...,ankle_3D_gyroscope_y,ankle_3D_gyroscope_z,ankle_3D_magnetometer_x,ankle_3D_magnetometer_y,ankle_3D_magnetometer_z,ankle_4D_orientation_x,ankle_4D_orientation_y,ankle_4D_orientation_z,ankle_4D_orientation_w,id
0,8.38,0,104.0,30.0000,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.009250,-0.017580,-61.1888,-38.95990,-58.143800,1.000000,0.000000,0.000000,0.000000,101
1,8.39,0,NaN,30.0000,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.004638,0.000368,-59.8479,-38.89190,-58.525300,1.000000,0.000000,0.000000,0.000000,101
2,8.40,0,NaN,30.0000,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.000148,0.022495,-60.7361,-39.41380,-58.399900,1.000000,0.000000,0.000000,0.000000,101
3,8.41,0,NaN,30.0000,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,-0.020301,0.011275,-60.4091,-38.76350,-58.395600,1.000000,0.000000,0.000000,0.000000,101
4,8.42,0,NaN,30.0000,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,-0.014303,-0.002823,-61.5199,-39.38790,-58.269400,1.000000,0.000000,0.000000,0.000000,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872528,100.19,0,NaN,25.1875,-4.71493,10.22250,4.66893,-5.04654,9.94944,4.50736,...,-0.062676,-0.127084,-46.5153,3.58240,-0.035995,0.598531,0.033615,0.799791,-0.031075,109
2872529,100.20,0,NaN,25.1875,-4.95932,10.37130,4.12594,-4.96890,10.29620,4.43102,...,-0.027006,-0.089808,-45.7474,3.54453,0.108583,0.598428,0.033012,0.799933,-0.030018,109
2872530,100.21,0,NaN,25.1875,-4.93997,9.83615,3.70468,-5.04613,10.35690,4.14405,...,-0.038024,-0.064709,-46.3997,4.22078,0.105504,0.598233,0.033172,0.800095,-0.029416,109
2872531,100.22,0,NaN,25.1875,-4.64941,9.11129,3.51904,-5.06854,9.75268,3.87359,...,-0.025796,-0.064357,-46.5282,4.48593,0.530240,0.598116,0.033427,0.800180,-0.029207,109


In [5]:
def fix_data(data):
    data = data.drop(data[data['activity_id']==0].index)
    data = data.interpolate()
    # fill all the NaN values in a coulmn with the mean values of the column
    for colName in data.columns:
        data[colName] = data[colName].fillna(data[colName].mean())
    activity_mean = data.groupby(['activity_id']).mean().reset_index()
    return data

data = fix_data(data)

In [6]:
data.describe()

,time_stamp,activity_id,heart_rate,hand_temperature,hand_3D_acceleration_16_x,hand_3D_acceleration_16_y,hand_3D_acceleration_16_z,hand_3D_acceleration_6_x,hand_3D_acceleration_6_y,hand_3D_acceleration_6_z,...,ankle_3D_gyroscope_y,ankle_3D_gyroscope_z,ankle_3D_magnetometer_x,ankle_3D_magnetometer_y,ankle_3D_magnetometer_z,ankle_4D_orientation_x,ankle_4D_orientation_y,ankle_4D_orientation_z,ankle_4D_orientation_w,id
count,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,...,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06,1.942872e+06
mean,1.705202e+03,8.081830e+00,1.074879e+02,3.275126e+01,-4.952973e+00,3.585079e+00,3.602232e+00,-4.886286e+00,3.573597e+00,3.786931e+00,...,-3.652767e-02,6.658552e-03,-3.158971e+01,1.396960e+00,1.725039e+01,3.812299e-01,-8.998437e-03,3.023855e-01,-5.698127e-02,1.045664e+02
std,1.093463e+03,6.174789e+00,2.699215e+01,1.794162e+00,6.239663e+00,6.893705e+00,3.956957e+00,6.245972e+00,6.589666e+00,3.942860e+00,...,6.383285e-01,2.012898e+00,1.834639e+01,2.168678e+01,1.970652e+01,3.036803e-01,5.714335e-01,3.326730e-01,4.800388e-01,2.333052e+00
min,3.120000e+01,1.000000e+00,5.700000e+01,2.487500e+01,-1.453670e+02,-1.043010e+02,-1.014520e+02,-6.121470e+01,-6.184170e+01,-6.193470e+01,...,-1.812690e+01,-1.401960e+01,-1.728650e+02,-1.379080e+02,-1.027160e+02,1.521280e-06,-9.568760e-01,-8.768380e-01,-9.972810e-01,1.010000e+02
25%,7.445400e+02,3.000000e+00,8.600000e+01,3.168750e+01,-8.969760e+00,1.058210e+00,1.161470e+00,-8.866490e+00,1.057690e+00,1.363370e+00,...,-1.066083e-01,-4.416998e-01,-4.171632e+01,-1.248465e+01,3.794668e+00,1.426340e-01,-6.129580e-01,4.914657e-03,-5.378710e-01,1.020000e+02
50%,1.480330e+03,6.000000e+00,1.040000e+02,3.312500e+01,-5.450280e+00,3.526550e+00,3.431410e+00,-5.378515e+00,3.567850e+00,3.662640e+00,...,-3.950165e-03,-2.327570e-03,-3.400820e+01,7.769370e-01,1.876755e+01,2.838390e-01,0.000000e+00,3.055330e-01,0.000000e+00,1.050000e+02
75%,2.663610e+03,1.300000e+01,1.240000e+02,3.406250e+01,-9.577087e-01,6.453505e+00,6.531522e+00,-9.058870e-01,6.458670e+00,6.775980e+00,...,1.162570e-01,9.181807e-02,-1.790600e+01,1.783892e+01,3.121160e+01,5.600202e-01,6.153723e-01,5.960633e-01,4.382870e-01,1.070000e+02
max,4.245680e+03,2.400000e+01,2.020000e+02,3.550000e+01,6.285960e+01,1.556990e+02,1.577600e+02,5.282140e+01,6.225980e+01,6.192340e+01,...,1.358820e+01,1.652880e+01,9.155160e+01,9.424780e+01,1.469000e+02,1.000000e+00,9.595380e-01,9.514820e-01,9.961050e-01,1.090000e+02


In [7]:
print('Size of the data: ', data.size)
print('Shape of the data: ', data.shape)
print('Number of columns in the data: ', len(data.columns))
result_id = data.groupby(['id']).mean().reset_index()
print('Number of uniqe ids in the data: ', len(result_id))
result_act = data.groupby(['activity_id']).mean().reset_index()
print('Numbe of uniqe activitys in the data: ',len(result_act))

Size of the data:  106857960
Shape of the data:  (1942872, 55)
Number of columns in the data:  55
Number of uniqe ids in the data:  9
Numbe of uniqe activitys in the data:  12


In [16]:
#remove unused columns
columns_dropped=[]
for i in data.columns:
    if "orientation" in i:
        columns_dropped.append(i)
data=data.drop(columns=columns_dropped)
data.shape

(1942872, 43)

In [17]:
#select the subjects and the activities
activity_list=[3,4,12,13]
no_id_list=[108,109]
data_4_activities= data[data.activity_id.isin(activity_list)] #filter is in pour considérer que les 4 activités
data_4_activities= data_4_activities[~data_4_activities.id.isin(no_id_list)] #filter is not in pour enlever les sujets 8 et 9
data_4_activities.shape

(572821, 43)

In [25]:
#normalization of data
def data_Normalization(data):
    df_std = data.copy()
    for column in df_std.columns[2:-1]:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
        
    return df_std
    
# call the z_score function
data_4_activities_normalized = data_Normalization(data_4_activities)

data_4_activities_normalized

,time_stamp,activity_id,heart_rate,hand_temperature,hand_3D_acceleration_16_x,hand_3D_acceleration_16_y,hand_3D_acceleration_16_z,hand_3D_acceleration_6_x,hand_3D_acceleration_6_y,hand_3D_acceleration_6_z,...,ankle_3D_acceleration_6_x,ankle_3D_acceleration_6_y,ankle_3D_acceleration_6_z,ankle_3D_gyroscope_x,ankle_3D_gyroscope_y,ankle_3D_gyroscope_z,ankle_3D_magnetometer_x,ankle_3D_magnetometer_y,ankle_3D_magnetometer_z,id
53595,544.33,3,-0.753143,-0.235209,2.421987,2.148492,0.234106,2.401764,2.041617,0.247049,...,-0.150665,-0.149875,0.564742,0.016557,0.183528,-0.025918,-2.497168,1.987377,-1.247671,101
53596,544.34,3,-0.753143,-0.235209,2.429124,2.104231,0.178823,2.411895,2.075928,0.225339,...,-0.150665,-0.147761,0.564728,-0.019428,0.080285,-0.020018,-2.470813,1.960201,-1.254361,101
53597,544.35,3,-0.753143,-0.235209,2.420058,2.046034,0.164894,2.415000,2.041416,0.209185,...,-0.147933,-0.147755,0.572969,0.007178,0.115703,-0.019174,-2.492081,2.002818,-1.235230,101
53598,544.36,3,-0.753143,-0.235209,2.412138,1.973805,0.206629,2.424620,1.978092,0.182265,...,-0.147910,-0.147750,0.581232,-0.005065,0.062022,-0.000212,-2.496685,1.945429,-1.260456,101
53599,544.37,3,-0.753143,-0.235209,2.464793,2.046375,0.263255,2.424618,1.949403,0.209357,...,-0.142512,-0.141411,0.572886,-0.048076,0.074185,-0.023561,-2.460927,2.012136,-1.229242,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380641,2388.07,4,-0.799673,0.060856,0.383572,1.390258,-1.285597,0.406513,1.455883,-1.283886,...,-0.198341,-0.078366,-0.034378,0.011823,0.089666,-0.014373,-1.224077,1.327160,0.436385,107
2380642,2388.08,4,-0.799673,0.060856,0.409024,1.433430,-1.299020,0.399627,1.415773,-1.278386,...,-0.190164,-0.065667,-0.018000,0.011745,0.120267,-0.009061,-1.235399,1.315543,0.464578,107
2380643,2388.09,4,-0.799673,0.060856,0.408120,1.433043,-1.340736,0.399714,1.444478,-1.289258,...,-0.182093,-0.063564,-0.038735,-0.005369,0.051080,-0.023216,-1.236235,1.327703,0.408122,107
2380644,2388.10,4,-0.799673,0.060856,0.407878,1.447462,-1.354686,0.409898,1.450110,-1.273067,...,-0.190221,-0.065680,-0.038655,-0.018920,0.103540,0.001282,-1.241680,1.321943,0.443384,107


In [108]:
def data_id_Preprocess(data_id):
    data=data_id.groupby(['activity_id'])
    data_dict=dict()
    for i,j in data:
        data_dict[i]=torch.tensor(j.drop(columns=['activity_id',"time_stamp"]).values)
    return data_dict

In [115]:
#la sortie de cette fonction est un dictionnaire sur les id vers un dictionnaire sur les activity_id vers un tensor 
#contenant toutes les données
def data_Preprocess(data):
    data_ids=data.groupby(['id'])
    data_dict=dict()
    for i,j in data_ids:
        data_dict[i]=data_id_Preprocess(j.drop(columns=['id']))
    return data_dict
data_processed=data_Preprocess(data_4_activities_normalized)
data_processed[101][3].shape


torch.Size([21717, 40])

In [121]:
def pytorch_rolling_window(x, window_size, step_size=1):
    return x.unfold(0,window_size,step_size)

In [194]:
def split_dataset(data,id_test):
    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    for i in data:
        if i!=id_test:
            for j in data[i]:
                se=pytorch_rolling_window(data[i][j], 256, step_size=128)
                y_train.append(torch.tensor([j]*se.shape[0]))
                #Exemple se.shape (nombre d'extraits, nb de channels, window length)
                x_train.append(se)
        else:
            for j in data[i]:
                se=pytorch_rolling_window(data[i][j], 256, step_size=128)
                y_test.append(torch.tensor([j]*se.shape[0]))
                #Exemple se.shape (nombre d'extraits, nb de channels, window length)
                x_test.append(se)
    x_train,y_train,x_test,y_test=torch.cat(x_train, dim=0),torch.cat(y_train, dim=0),torch.cat(x_test, dim=0),torch.cat(y_test, dim=0)
    idx_train=torch.randperm(x_train.shape[0])
    idx_test=torch.randperm(x_test.shape[0])
    return  x_train[idx_train],y_train[idx_train],x_test[idx_test],y_test[idx_test]

x_train,y_train,x_test,y_test=split_dataset(data_processed,103)

In [195]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

(torch.Size([3851, 40, 256]),
 torch.Size([3851]),
 torch.Size([582, 40, 256]),
 torch.Size([582]))